In [7]:
import numpy as np
from scipy import optimize, special
import itertools
from tqdm import tqdm # cool load bar


In [2]:
def initialize_weights(L_in, L_out):
    dt = np.dtype('f8')   # 64-bit floating-point number
    t = np.zeros((L_out, L_in + 1), dtype=dt)
    e = 0.12 # sigma
    t[:] = np.random.randn(*t.shape) * 2.0 * e - e